# Predicting Crop Yield using SVM with and without Dimensionality Reduction

## Introduction
In this notebook, I will be using the USDA NASS Crop Yield Data to predict crop yield using SVM. I will be using the 7 similarity scoring metrics that I have created to reduce the dimensionality of the data. Since I have similarity scores measure similarity between 2 different geo locations, for each of their features, I will use each individual features sim score as an output for the yield, as well as the average of all the sim scores features for the output of the yield. I will do this across all 7 sim scores. Then I will do an average of all sim scores. Then I will analyze this to see if there is better predictive ability from that than to a regular larger model trained on the higher dimensionality data.


In [12]:
# import all nessecary libraries
import sklearn
import numpy as np
import pandas as pd
from sklearn import svm
from plotly import express as px
from plotly import graph_objects as go
from sklearn.model_selection import train_test_split

print("Succesfully imported all libraries")

Succesfully imported all libraries


In [50]:
# load the data

# a combination dataframe of the data from the weather dataset and yield dataset
combined_df = pd.read_csv('../../data/pecan/combined_05_23_24.csv')

# a combined dataframe of all the similarity scoring data
similarity_df = pd.read_csv('../../data/pecan/dataframes/combined_df_similarity_scores.csv')

# a dataframe of the crop yield
yield_df = pd.read_csv('../../data/pecan/yield_05_23_24.csv')

## View the Data

In [23]:
combined_df.shape

(192840, 17)

In [24]:
similarity_df.shape

(42, 15)

In [26]:
combined_df.head(10)

,city_name,lat,lon,temp,dew_point,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,clouds_all,weather_main,weather_description,date,yield
0,Doña Ana,32.39308,-106.81578,8.99,1.42,7.93,7.60,9.46,1019.0,59.0,2.10,20.0,90.0,1.0,16.0,2016-01-01 00:00:00+00:00,1837.0
1,Doña Ana,32.39308,-106.81578,8.25,2.07,7.63,7.44,8.68,1020.0,65.0,1.52,51.0,87.0,1.0,16.0,2016-01-01 01:00:00+00:00,1837.0
2,Doña Ana,32.39308,-106.81578,7.37,1.44,6.04,6.10,7.80,1021.0,66.0,2.10,340.0,81.0,1.0,0.0,2016-01-01 02:00:00+00:00,1837.0
3,Doña Ana,32.39308,-106.81578,5.58,1.32,3.13,4.30,6.02,1022.0,74.0,3.10,350.0,40.0,1.0,21.0,2016-01-01 03:00:00+00:00,1837.0
4,Doña Ana,32.39308,-106.81578,5.43,0.79,5.43,4.75,5.90,1023.0,72.0,1.17,48.0,87.0,1.0,16.0,2016-01-01 04:00:00+00:00,1837.0
5,Doña Ana,32.39308,-106.81578,4.68,0.45,4.68,3.70,5.29,1024.0,74.0,0.00,0.0,85.0,1.0,16.0,2016-01-01 05:00:00+00:00,1837.0
6,Doña Ana,32.39308,-106.81578,3.89,0.23,3.89,3.27,4.51,1025.0,77.0,0.00,0.0,91.0,1.0,16.0,2016-01-01 06:00:00+00:00,1837.0
7,Doña Ana,32.39308,-106.81578,3.06,-1.15,0.10,2.44,3.70,1025.0,73.0,3.10,270.0,40.0,1.0,21.0,2016-01-01 07:00:00+00:00,1837.0
8,Doña Ana,32.39308,-106.81578,2.65,-1.50,0.02,2.09,3.36,1026.0,73.0,2.60,300.0,0.0,0.0,25.0,2016-01-01 08:00:00+00:00,1837.0
9,Doña Ana,32.39308,-106.81578,2.25,-1.36,0.05,1.87,2.94,1027.0,76.0,2.10,290.0,40.0,1.0,21.0,2016-01-01 09:00:00+00:00,1837.0


In [36]:
similarity_df = similarity_df.rename(columns={'Unnamed: 0': 'city_names'})
similarity_df.head(10)

,city_names,lat,lon,temp,dew_point,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,clouds_all,weather_id,Similarity Type
0,Doña Ana-Chaves County,681.651657,1475.458784,1.953988e+03,3.106781e+03,2.171000e+03,1.916711e+03,2.039767e+03,1.921502e+03,1.087209e+04,1592.502058,8.106963e+04,2.390010e+04,5.529241e+04,Euclidean
1,Doña Ana-Otero County,151.793161,769.031368,4.539151e+03,2.890086e+03,4.881963e+03,4.688181e+03,4.339047e+03,1.988777e+03,1.181816e+04,1196.956873,7.386639e+04,2.213712e+04,5.921987e+04,Euclidean
2,Doña Ana-Sierra County,466.864816,179.698079,1.357260e+03,2.585302e+03,1.593666e+03,1.334880e+03,1.418370e+03,6.219622e+02,6.646865e+03,1414.385875,8.142685e+04,1.901853e+04,4.741269e+04,Euclidean
3,Chaves County-Otero County,529.858496,706.427416,4.014002e+03,3.137783e+03,4.317280e+03,4.291811e+03,3.715638e+03,2.202131e+03,1.087521e+04,1447.905650,7.552922e+04,2.282695e+04,5.997001e+04,Euclidean
4,Chaves County-Sierra County,214.786840,1655.156863,1.858861e+03,3.763669e+03,2.065357e+03,1.873333e+03,1.880476e+03,1.810586e+03,1.163909e+04,1542.782991,8.392373e+04,2.380490e+04,5.433247e+04,Euclidean
5,Otero County-Sierra County,315.071656,948.729447,3.567178e+03,2.955140e+03,3.766204e+03,3.823967e+03,3.319337e+03,1.868933e+03,1.325814e+04,1330.095701,8.075515e+04,2.428489e+04,6.047739e+04,Euclidean
6,Doña Ana-Chaves County,429174.292464,928962.724008,9.317640e+05,1.385963e+06,9.923937e+05,9.067831e+05,9.791663e+05,8.977060e+05,5.034805e+06,771617.400000,3.668089e+07,9.198797e+06,1.035451e+07,Manhattan
7,Doña Ana-Otero County,95570.401128,484189.380336,2.736576e+06,1.456895e+06,2.941011e+06,2.838063e+06,2.603848e+06,1.018373e+06,5.927118e+06,560046.860000,3.057681e+07,8.716527e+06,1.218218e+07,Manhattan
8,Doña Ana-Sierra County,293942.478120,113139.600096,7.306495e+05,1.246349e+06,8.497631e+05,7.004886e+05,7.628089e+05,2.702870e+05,3.006084e+06,655863.560000,3.534082e+07,6.607196e+06,7.407486e+06,Manhattan
9,Chaves County-Otero County,333603.891336,444773.343672,2.302916e+06,1.568665e+06,2.480981e+06,2.477631e+06,2.113210e+06,1.173129e+06,5.447879e+06,715409.720000,3.334995e+07,8.923066e+06,1.272044e+07,Manhattan


In [51]:
yield_df.head(10)

,Year,State,Ag District,Ag District Code,County,County ANSI,Commodity,Value
0,2022,NEW MEXICO,SOUTHEAST,90,Chaves County,5.0,PECANS,"1,500"
1,2022,NEW MEXICO,SOUTHEAST,90,Doña Ana,13.0,PECANS,"1,610"
2,2022,NEW MEXICO,SOUTHEAST,90,Otero County,35.0,PECANS,"1,560"
3,2022,NEW MEXICO,SOUTHWEST,70,Sierra County,51.0,PECANS,"2,360"
4,2021,NEW MEXICO,SOUTHEAST,90,Doña Ana,13.0,PECANS,"1,850"
5,2021,NEW MEXICO,SOUTHWEST,70,Sierra County,51.0,PECANS,"2,300"
6,2020,NEW MEXICO,SOUTHEAST,90,Chaves County,5.0,PECANS,"2,300"
7,2020,NEW MEXICO,SOUTHEAST,90,Doña Ana,13.0,PECANS,"1,780"
8,2020,NEW MEXICO,SOUTHEAST,90,Otero County,35.0,PECANS,"1,350"
9,2019,NEW MEXICO,SOUTHEAST,90,Chaves County,5.0,PECANS,"2,380"


## Creating a new df for our x and y values

I am taking all of the similarity scores and making those the colunms that we will grab from the df to make our x values. Then I will take the yield as the y value. that will match up to the sim scores. 
For exampe the x from a euclidean sim score will be matched up with the yield for that sim score for each of the geo locations it is matched up with

In [53]:
yield_df.values.flatten()
yield

array([2022, 'NEW MEXICO', 'SOUTHEAST', 90, 'Chaves County', 5.0,
       'PECANS', '1,500', 2022, 'NEW MEXICO', 'SOUTHEAST', 90, 'Doña Ana',
       13.0, 'PECANS', '1,610', 2022, 'NEW MEXICO', 'SOUTHEAST', 90,
       'Otero County', 35.0, 'PECANS', '1,560', 2022, 'NEW MEXICO',
       'SOUTHWEST', 70, 'Sierra County', 51.0, 'PECANS', '2,360', 2021,
       'NEW MEXICO', 'SOUTHEAST', 90, 'Doña Ana', 13.0, 'PECANS', '1,850',
       2021, 'NEW MEXICO', 'SOUTHWEST', 70, 'Sierra County', 51.0,
       'PECANS', '2,300', 2020, 'NEW MEXICO', 'SOUTHEAST', 90,
       'Chaves County', 5.0, 'PECANS', '2,300', 2020, 'NEW MEXICO',
       'SOUTHEAST', 90, 'Doña Ana', 13.0, 'PECANS', '1,780', 2020,
       'NEW MEXICO', 'SOUTHEAST', 90, 'Otero County', 35.0, 'PECANS',
       '1,350', 2019, 'NEW MEXICO', 'SOUTHEAST', 90, 'Chaves County', 5.0,
       'PECANS', '2,380', 2019, 'NEW MEXICO', 'SOUTHEAST', 90, 'Doña Ana',
       13.0, 'PECANS', '2,010', 2019, 'NEW MEXICO', 'SOUTHEAST', 90,
       'Otero County'